In [12]:
#default_exp tests

In [13]:
#hide
%load_ext autoreload
%autoreload 2    
%load_ext line_profiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [14]:
#export
from pathlib import Path
from datetime import timedelta, datetime
from collections import defaultdict, namedtuple
from typing import *
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from rfpy.main import parse_input
from rfpy.parser import get_files
from rfpy.blocks import *
from rfpy.utils import *
from nbdev.showdoc import *
from fastcore.xtras import parallel
from fastcore.foundation import L
from fastcore.test import *

Com o objetivo de estudar o código fui reescrevendo-o. Tentar reescrever funções é um bom exercício para entender e internalizar a implementação utilizada. Assim foi possível identificar várias oportunidades para simplesmente refatorar e simplificar o código. 

Utilizarei várias funcionalidades da biblioteca [fastcore](https://fastcore.fast.ai/basics.html), esta  extende de maneira muito eficiente várias funcionalidades básicas do python e assim não precisamos _reiventar a roda_.

### `config.tradutor.yaml` 

Como só temos 2 parâmetros, `entrada` e `saída` escolhi não utilizar um arquivo de configuração por não achar ser necessário. Outro benefício dessa abordagem é utilizar as variáveis que antes estavam no arquivo de configuração como parâmetros de entrada numa linha de comando.

Como não utilizei um arquivo de configuração não é necessário chamar a função `busca_arquivos` do arquivo `helper.py`

In [15]:
VERBOSE = True
entrada = Path(r'D:\OneDrive - ANATEL\Backup_Rfeye_SP\RPO\PMEC2020\Ribeirao_Preto_SP\SLMA')
saida = Path(r'C:\Users\rsilva\Downloads')

In [16]:
show_doc(get_files)

<h4 id="get_files" class="doc_header"><code>get_files</code><a href="https://github.com/ronaldokun/rfpy/tree/master/rfpy/parser.py#L17" class="source_link" style="float:right">[source]</a></h4>

> <code>get_files</code>(**`path`**, **`extensions`**=*`None`*, **`recurse`**=*`True`*, **`folders`**=*`None`*, **`followlinks`**=*`True`*)

Get all the files in `path` with optional `extensions`, optionally with `recurse`, only in `folders`, if specified.

In [17]:
arquivos = get_files(entrada) ; arquivos

(#255) [Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200317_232233.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200318_225525.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200318_225535.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200318_225556.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200318_225615.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200410_183001.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200423_162327.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/

O Objeto retornado `L` é uma extensão da lista python com funcionalidades adicionais, uma delas como  podemos ver é que a representaçõa da lista impressa vem com o número de arquivos da lista. 
Temos 255 arquivos bin na pasta entrada. Podemos filtrar por pasta também

In [18]:
arquivos_bin = get_files(entrada, extensions=['.bin'], folders='tmp')

In [19]:
arquivos_bin

(#1) [Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/tmp/rfeye002304_SLMA_bimestral_PEAK_200829_234902.bin')]

O Código executado em `tradutor_RFEYE.py` está totalmente contido de forma mais geral em `main.py`. Chamei de main justamente porque esse será o arquivo base que chama outros arquivos e funções.

In [35]:
%%time
save_file_a = saida / 'file_a.csv'
save_file_b = saida / 'file_b.csv'

COLS_A = ['Block_Number', 'Latitude', 'Longitude', 'Altitude', 'Initial_Time',
     'Sample_Duration', 'Start_Frequency', 'Stop_Frequency', 'Vector_Length', 
     'Trace_Type', 'Antenna_Type', 'Equipement_ID']

COLS_B = ['Block_Number', 'Frequency(HZ)', 'Nivel(dBm)']

BLOCO_40 = ['latitude', 'longitude', 'altitude']

BLOCO_63 = ['datetime_stamp',
           'spent_time_microsecs', 'start_mega', 'stop_mega', 'data_points', 
           'processing', 'id_antenna']

def file2block(file: str):
    map_block = defaultdict(L)
    for bloco in L(binary_file_generator(file)).map(create_base_block):
        btype = bloco.type
        map_block[btype].append(block_constructor(btype, bloco))
    return map_block


def extract_meta_bin(file: str, outfile: str):
    map_block = file2block(file)
    n_data = max(len(map_block[40]), len(map_block[63]))
    output = {}
    output['Block_Number'] = L(range(n_data)) 
    for attr in BLOCO_40:
        output[attr] = map_block[40].attrgot(attr)
    for attr in BLOCO_63:
        output[attr] = map_block[63].attrgot(attr)            
    output['Equipement_ID'] = L(getattr(map_block[21][0], 'hostname')) * n_data    
    df = pd.DataFrame(output)    
    df.columns = COLS_A    
    df.to_csv(outfile, index=False)    
    return df
    
def extract_espectro(file: str):
    pass
    

Wall time: 0 ns


In [36]:
df = extract_meta_bin(arquivos_bin[0])
df.head()

,Block_Number,Latitude,Longitude,Altitude,Initial_Time,Sample_Duration,Start_Frequency,Stop_Frequency,Vector_Length,Trace_Type,Antenna_Type,Equipement_ID
0,0,-21.228926,-47.759933,619.6,2020-08-29 23:50:01.059268,38373,108,137,14848,peak,1,rfeye002304
1,1,-21.228928,-47.759912,615.6,2020-08-29 23:51:00.579360,38529,108,137,14848,peak,1,rfeye002304
2,2,-21.228921,-47.759922,612.0,2020-08-29 23:52:01.969255,38517,108,137,14848,peak,1,rfeye002304
3,3,-21.228934,-47.759922,617.2,2020-08-29 23:53:01.359546,38463,108,137,14848,peak,1,rfeye002304
4,4,-21.228931,-47.759926,624.0,2020-08-29 23:54:00.959245,38372,108,137,14848,peak,1,rfeye002304
